# Reflex Agent with Vertex AI (Gemini)
This notebook demonstrates a simple **reflexion** loop: attempt → critique → revise → repeat until valid.
Artifacts are saved to Cloud Storage for grading.

In [ ]:
import os, json, time, uuid
from typing import Any
from pydantic import BaseModel, ValidationError
from google.cloud import storage, aiplatform
from vertexai.generative_models import GenerativeModel

PROJECT_ID = os.getenv('PROJECT_ID') or input('Enter PROJECT_ID: ')
REGION = os.getenv('REGION','us-central1')
BUCKET = os.getenv('BUCKET') or input('Enter BUCKET (no gs://): ')
RUN_ID = uuid.uuid4().hex[:8]
RUN_PREFIX = f'agent_runs/{RUN_ID}'
LATEST_PREFIX = 'agent_runs/latest'

aiplatform.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel('gemini-1.5-pro')
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET)
print('Using', PROJECT_ID, REGION, BUCKET, RUN_ID)

In [ ]:
# Rubric + schema using Pydantic
class Step(BaseModel):
    name: str
    description: str
    inputs: list[str]
    outputs: list[str]

class Plan(BaseModel):
    steps: list[Step]
    destination_bq: str

SYSTEM_RUBRIC = f'''
Return ONLY JSON matching this exact schema:
{
  "steps": [
    {"name": "extract", "description": "...", "inputs": ["..."], "outputs": ["..."]},
    {"name": "transform", "description": "...", "inputs": ["..."], "outputs": ["..."]},
    {"name": "load", "description": "...", "inputs": ["..."], "outputs": ["..."]}
  ],
  "destination_bq": "{PROJECT_ID}.demo.sales_etl"
}
Constraints:
- Exactly 3 steps in this order: extract → transform → load.
- Each step has non-empty fields and at least one input & output.
- destination_bq must start with your PROJECT_ID.
If you make mistakes, you'll receive a critique. Use it to correct the JSON. Always return pure JSON.
'''

In [ ]:
def write_gcs(path: str, payload: Any):
    blob = bucket.blob(path)
    if isinstance(payload, (dict, list)):
        payload = json.dumps(payload, indent=2)
    blob.upload_from_string(str(payload), content_type='application/json')

def attempt() -> str:
    prompt = (
        'Produce the JSON plan per the rubric. Scenario: Read daily CSV from GCS, '
        'clean column names and filter out negative price, then load to BigQuery.'
    )
    return model.generate_content([SYSTEM_RUBRIC, prompt]).text

def critique(json_text: str) -> str:
    return model.generate_content([
        SYSTEM_RUBRIC,
        json_text,
        'Critique strictly against the rubric. If valid, respond ONLY "VALID"; else list minimal issues.'
    ]).text

def revise(json_text: str, critique_text: str) -> str:
    return model.generate_content([
        SYSTEM_RUBRIC,
        json_text,
        critique_text,
        'Correct the JSON per the critique. Output ONLY corrected JSON.'
    ]).text

def validate_plan(json_text: str) -> tuple[bool, str]:
    try:
        data = json.loads(json_text)
        plan = Plan(**data)
    except (json.JSONDecodeError, ValidationError) as e:
        return False, f'Schema/JSON error: {e}'
    names = [s.name.lower() for s in plan.steps]
    if names != ['extract','transform','load']:
        return False, f'Wrong step order: {names}'
    if not plan.destination_bq.startswith(f'{PROJECT_ID}.'):
        return False, 'destination_bq must start with PROJECT_ID.'
    return True, 'OK'

In [ ]:
MAX_ITERS = 4
history = []
current = attempt()
final = {"status": "FAILED", "run_id": RUN_ID}

for i in range(1, MAX_ITERS+1):
    valid, reason = validate_plan(current)
    history.append({"iter": i, "valid": valid, "reason": reason})
    write_gcs(f"agent_runs/{RUN_ID}/iter_{i}.json", {"valid": valid, "reason": reason, "candidate": current})
    if valid:
        final = {"status": "SUCCESS", "run_id": RUN_ID, "iters": i}
        break
    fb = critique(current)
    write_gcs(f"agent_runs/{RUN_ID}/iter_{i}_critique.json", {"critique": fb})
    current = revise(current, fb)

write_gcs(f"agent_runs/{RUN_ID}/final.json", {"history": history, "final": current, "final_status": final})
write_gcs("agent_runs/latest/final.json", {"history": history, "final": current, "final_status": final})
if final["status"] == "SUCCESS":
    write_gcs(f"agent_runs/{RUN_ID}/success.json", final)
    write_gcs("agent_runs/latest/success.json", final)
final